In [17]:
import sys, os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, Catalog
from pyspark.sql import DataFrame, DataFrameStatFunctions, DataFrameNaFunctions
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.types import Row
from subprocess import check_output

SPARK_DRIVER_HOST = check_output(["hostname", "-i"]).decode(encoding="utf-8").strip()
spark_conf = SparkConf()
spark_conf.setAll([
    ('spark.master', 'spark://spark:7077'),
    ('spark.app.name', 'myApp'),
    ('spark.submit.deployMode', 'client'),
    ('spark.ui.showConsoleProgress', 'true'),
    ('spark.eventLog.enabled', 'false'),
    ('spark.logConf', 'false'),
    ('spark.driver.bindAddress', '0.0.0.0'),
    ('spark.driver.host', SPARK_DRIVER_HOST),
    ('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0'),
    ("spark.hadoop.fs.s3a.endpoint", 'http://minio:9000'),
    ('spark.hadoop.fs.s3a.access.key', 'minio-root-user'),
    ('spark.hadoop.fs.s3a.secret.key', 'minio-root-password'),
    ('spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled', True),
    ("spark.hadoop.fs.s3a.fast.upload", True),
    ("spark.hadoop.fs.s3a.path.style.access", True),
    ("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
])
 
spark_sess          = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark_ctxt          = spark_sess.sparkContext
spark_reader        = spark_sess.read
spark_streamReader  = spark_sess.readStream
spark_ctxt.setLogLevel("WARN")

citiesDF = spark_sess.read.option("header",True).csv('s3a://test-001/cities.csv')

citiesDF.show(truncate=False)


22/09/28 23:53:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
+-----+-------+-------+-----+-------+-------+-------+-----+------------------+--------+
|LatD | "LatM"| "LatS"| "NS"| "LonD"| "LonM"| "LonS"| "EW"| "City"           | "State"|
+-----+-------+-------+-----+-------+-------+-------+-----+------------------+--------+
|   41|    5  |   59  | "N" |     80|   39  |    0  | "W" | "Youngstown"     | OH     |
|   42|   52  |   48  | "N" |     97|   23  |   23  | "W" | "Yankton"        | SD     |
|   46|   35  |   59  | "N" |    120|   30  |   36  | "W" | "Yakima"         | WA     |
|   42|   16  |   12  | "N" |     71|   48  |    0  | "W" | "Worcester"      | MA     |
|   43|   37  |   48  | "N" |     89|   46  |   11  | "W" | "Wisconsin Dells"| WI     |
|   36|    5  |   59  | "N" |     80|   15  |    0  | "W" | "Winston-Salem"  | NC     |
|   49|   52  |   48  | "N" |     97|    9  |    0  | "W" | "Winnipeg"       | MB     |


In [18]:
citiesDF.columns

['LatD',
 ' "LatM"',
 ' "LatS"',
 ' "NS"',
 ' "LonD"',
 ' "LonM"',
 ' "LonS"',
 ' "EW"',
 ' "City"',
 ' "State"']

In [19]:
cleanCitiesDF = citiesDF \
   .withColumnRenamed(' "LatM"', 'LatM') \
   .withColumnRenamed(' "LonM"', 'LonM') \
   .select('LatM', 'LonM')
cleanCitiesDF

DataFrame[LatM: string, LonM: string]

In [21]:
cleanCitiesDF.write.format("csv").save("s3a://test-001/cleanCities3.csv")

22/09/28 23:53:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
